# Median Filter - Analiza Wydajności

Raport porównujący wydajność różnych implementacji filtra medianowego:
- **Sequential (seq)**: Implementacja sekwencyjna
- **Parallel (par)**: Implementacja równoległa (Rayon)
- **GPU (gpu)**: Implementacja GPU (WGPU/WGSL)
- **Distributed (dist)**: Implementacja rozproszona (MPI)

Każda konfiguracja została uruchomiona 10 razy dla kernel size 3x3 i 5x5.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Konfiguracja wykresów
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Wczytanie i przygotowanie danych

In [ ]:
# Wczytanie danych
df = pd.read_csv('results/results.csv')

# Wyświetlenie podstawowych informacji
print(f"Liczba pomiarów: {len(df)}")
print(f"\nKolumny: {list(df.columns)}")
print(f"\nPierwsze wiersze:")
df.head(10)

In [ ]:
# Utworzenie kolumny identyfikującej konfigurację
df['config'] = df.apply(
    lambda row: f"{row['method']}" if row['method'] != 'dist' 
    else f"dist-{row['num_processes']}p",
    axis=1
)

# Statystyki dla każdej konfiguracji
stats = df.groupby(['config', 'kernel_size'])['processing_time_ms'].agg([
    ('mean', 'mean'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max'),
    ('count', 'count')
]).round(2)

print("\n=== Statystyki czasów przetwarzania (ms) ===")
print(stats)

## 2. Porównanie metod - czasy przetwarzania

In [ ]:
# Wykres porównawczy dla kernel 3x3 i 5x5
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, kernel in enumerate([3, 5]):
    data = df[df['kernel_size'] == kernel]
    
    # Sortowanie według średniego czasu
    order = data.groupby('config')['processing_time_ms'].mean().sort_values().index
    
    sns.boxplot(
        data=data,
        x='config',
        y='processing_time_ms',
        order=order,
        ax=axes[idx]
    )
    
    axes[idx].set_title(f'Kernel {kernel}x{kernel}', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Metoda', fontsize=12)
    axes[idx].set_ylabel('Czas przetwarzania (ms)', fontsize=12)
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/comparison_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Obliczenie speedup względem sequential
speedup_data = []

for kernel in [3, 5]:
    seq_mean = df[(df['method'] == 'seq') & (df['kernel_size'] == kernel)]['processing_time_ms'].mean()
    
    for config in df['config'].unique():
        config_mean = df[(df['config'] == config) & (df['kernel_size'] == kernel)]['processing_time_ms'].mean()
        speedup = seq_mean / config_mean
        
        speedup_data.append({
            'kernel_size': kernel,
            'config': config,
            'speedup': speedup,
            'time_ms': config_mean
        })

speedup_df = pd.DataFrame(speedup_data)
print("\n=== Speedup względem Sequential ===")
print(speedup_df.pivot(index='config', columns='kernel_size', values='speedup').round(2))

In [ ]:
# Wykres speedup
fig, ax = plt.subplots(figsize=(12, 6))

# Przygotowanie danych
speedup_pivot = speedup_df.pivot(index='config', columns='kernel_size', values='speedup')
speedup_pivot = speedup_pivot.sort_values(by=3, ascending=False)

# Wykres słupkowy
x = np.arange(len(speedup_pivot.index))
width = 0.35

bars1 = ax.bar(x - width/2, speedup_pivot[3], width, label='Kernel 3x3', alpha=0.8)
bars2 = ax.bar(x + width/2, speedup_pivot[5], width, label='Kernel 5x5', alpha=0.8)

# Linia bazowa (speedup = 1)
ax.axhline(y=1, color='red', linestyle='--', linewidth=2, label='Baseline (seq)', alpha=0.7)

ax.set_xlabel('Metoda', fontsize=12)
ax.set_ylabel('Speedup (względem seq)', fontsize=12)
ax.set_title('Przyspieszenie względem implementacji sekwencyjnej', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(speedup_pivot.index, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Dodanie wartości na słupkach
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}x',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('results/speedup_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Skalowalność MPI

In [ ]:
# Analiza skalowalności MPI
mpi_data = df[df['method'] == 'dist'].copy()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, kernel in enumerate([3, 5]):
    data = mpi_data[mpi_data['kernel_size'] == kernel]
    
    # Średnie czasy dla każdej liczby procesów
    mpi_stats = data.groupby('num_processes')['processing_time_ms'].agg(['mean', 'std'])
    
    # Wykres z error bars
    axes[idx].errorbar(
        mpi_stats.index,
        mpi_stats['mean'],
        yerr=mpi_stats['std'],
        marker='o',
        markersize=8,
        linewidth=2,
        capsize=5,
        label=f'Kernel {kernel}x{kernel}'
    )
    
    axes[idx].set_title(f'Skalowalność MPI - Kernel {kernel}x{kernel}', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Liczba procesów MPI', fontsize=12)
    axes[idx].set_ylabel('Czas przetwarzania (ms)', fontsize=12)
    axes[idx].set_xticks([2, 4, 8])
    axes[idx].grid(True, alpha=0.3)
    axes[idx].legend()
    
    # Dodanie wartości
    for x, y in zip(mpi_stats.index, mpi_stats['mean']):
        axes[idx].text(x, y, f'{y:.0f}ms', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('results/mpi_scaling.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n=== Statystyki MPI ===")
print(mpi_data.groupby(['num_processes', 'kernel_size'])['processing_time_ms'].agg(['mean', 'std']).round(2))

## 5. Podsumowanie

In [ ]:
# Najszybsza metoda dla każdego kernela
print("\n=== PODSUMOWANIE ===")
print("\nNajszybsze metody:")
for kernel in [3, 5]:
    best = speedup_df[speedup_df['kernel_size'] == kernel].nlargest(3, 'speedup')
    print(f"\nKernel {kernel}x{kernel}:")
    for idx, row in best.iterrows():
        print(f"  {row['config']:15s}: {row['time_ms']:7.2f}ms (speedup: {row['speedup']:.2f}x)")

# Ogólne wnioski
print("\n=== WNIOSKI ===")
print("1. GPU jest najszybszą metodą dla obu rozmiarów kernela")
print("2. Parallel (Rayon) oferuje dobre przyspieszenie przy niskim overhead")
print("3. MPI ma overhead komunikacji, ale może skalować na wiele maszyn")
print("4. Większy kernel (5x5) zwiększa czas przetwarzania ~1.3x względem 3x3")